In [28]:
import pandas as pd
import numpy as np

#biblioteca plotly
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [29]:
dados_nordeste = pd.read_csv("dados/enade2017_computacao_nordeste_convertido.csv")
# dados_nordeste = dados_nordeste[dados_nordeste["TP_INSCRICAO"]==1] desnecessario
dados_custo = pd.read_csv("dados/instituicoes_federais_nordeste_custo.csv")[["Instituicao", "custoAluno"]]

dados_custo = pd.merge( 
                        dados_nordeste, 
                        dados_custo, 
                        left_on=['Instituicao'], 
                        right_on=['Instituicao'], 
                        how='inner'
                      )


### A nota geral do aluno no ENADE é influenciada por características socioeconômicas do mesmo?


In [30]:
dados_nordeste_publicas = dados_nordeste[ dados_nordeste["QE_I08_Convertido"] != "Não informado" ]


order_q08 = pd.DataFrame(
                dados_nordeste_publicas.groupby('QE_I08_Convertido')['NT_GER'].mean()
            ).reset_index().sort_values(by='NT_GER', ascending=False)

layout = go.Layout(
            title=go.layout.Title(
                text='Desempenho dos alunos de acordo com suas caracteristicas Socioeconômicas',
                x=0.5
            ),
            yaxis=go.layout.YAxis(
                title=go.layout.yaxis.Title(
                    text='Média geral das notas',
                )
            ),
            width=700
        )

fig = go.Figure(layout=layout)
fig.add_trace(
    go.Bar(
            x=order_q08["QE_I08_Convertido"],
            y=order_q08["NT_GER"],
            marker_color=["#F08080", "#AFEEEE","#D8BFD8","#FFC0CB","#4169E1", "#D3D3D3", "#B0C4DE"],
    )
)


iplot(fig)

In [31]:
test = pd.DataFrame(
                dados_nordeste_publicas.groupby('QE_I08_Convertido').size()
            ).reset_index()
test

,QE_I08_Convertido,0
0,"1,5 a 3 salários-minimos",415
1,10 a 30 salários-minimos,183
2,"3 a 4,5 salários-minimos",337
3,"4,5 a 6 salários-minimos",173
4,6 a 10 salários-minimos,236
5,Acima de 30 salários,17
6,"Até 1,5 salários-minimos",305


#### Conclusão:
Sim, a nota é influenciada por características socioeconômicas dos mesmos. 

Pois, como podemos observar, os alunos que declararam ter uma renda familiar de **10 a 30 salários mínimos** conseguiram uma pontuação em média **10 pontos** maior do que os candidatos com renda familiar de até **1,5 salários minimos**.

Além disso, é possível observar que **a média e mediana das notas dos candidatos cresce conforme a classe socioeconômica do mesmo**. Ou seja, quanto maior é a renda familiar declarada, maior é a nota média daquele grupo de candidatos.

Não sendo o caso apenas para os candidatos que declararam renda familiar **Acima de 30 salários**. 
Talvez isso seja por conta do tamanho do grupo aqui representado, que seriam os alunos dentro dessa faixa socioeconômica *(17 candidatos)*, que pode não ser uma amostra grande o suficiente para representar, em termos gerais, o estrato social a que pertencem. 

### A nota geral do ENADE dos cursos é influenciada pelo custo aluno? Isto é, quanto mais caro o aluno, mais investimento é feito e maior é a nota?

In [32]:
# Ordenado do maior gasto para o menor gasto
dados_custo['custoAluno_legenda'] = ["Custo - " + str(item) for item in dados_custo['custoAluno']]
dados_custo_filtrado = pd.DataFrame(dados_custo.groupby('custoAluno_legenda')['NT_GER'].mean()).reset_index(
        ).sort_values(by='custoAluno_legenda', ascending=False)
layout = go.Layout(
    title=go.layout.Title(
        text='Desempenho de alunos de acordo com o investimento recebido',
        x=0.5
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Média geral das notas',
            )
    ),
    showlegend=False)

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
            x=dados_custo_filtrado["custoAluno_legenda"],
            y=dados_custo_filtrado["NT_GER"],
            marker_color=["#F08080", "#AFEEEE","#D8BFD8","#FFC0CB","#4169E1", "#D3D3D3", "#B0C4DE",
                         "#F08080", "#AFEEEE","#D8BFD8","#FFC0CB","#4169E1", "#D3D3D3", "#B0C4DE"],
            
    ))
iplot(fig)

In [33]:

layout = go.Layout(
    title=go.layout.Title(
        text='Desempenho de alunos de acordo com o investimento recebido',
        x=0.5
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Nota geral obtida',
            )
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Investimento dado ao aluno',
            )
    ),
    showlegend=False)

fig = go.Figure(layout=layout)

fig.add_trace(go.Scatter(x=dados_custo["custoAluno"], y=dados_custo["NT_GER"],
                    mode='markers',
                    name='markers'))

fig.show()

#### Conclusão:
Não é uma unanimidade, já que se compararmos o menor investimento poderemos ver a diferença de 10 pontos em média  para o de maior investimento, mas também se olharmos que as médias de notas para os candidatos de custo de **R\$51.548,10** e **R\$39.754,90** tem quase o mesmo desempenho, com uma diferença de quase **11 mil reais** de um investimento para outro.

### O desempenho dos alunos de universidade privada foram melhores do que o de universidades públicas? Para alunos com mesma características socioeconômicas

In [34]:
dados_nordeste_publicas = dados_nordeste[(dados_nordeste["CO_CATEGAD"]== 1)].append(dados_nordeste[(dados_nordeste["CO_CATEGAD"]== 2)]).append(dados_nordeste[(dados_nordeste["CO_CATEGAD"]== 3)])
dados_nordeste_publicas = dados_nordeste_publicas[dados_nordeste_publicas["QE_I08_Convertido"] != "Não informado"]
order_q08_publicas = pd.DataFrame(dados_nordeste_publicas.groupby('QE_I08_Convertido')['NT_GER'].mean()).reset_index(
        ).sort_values(by='NT_GER', ascending=False)

dados_nordeste_privadas = dados_nordeste[(dados_nordeste["CO_CATEGAD"]== 4)].append(dados_nordeste[(dados_nordeste["CO_CATEGAD"]== 5)])
dados_nordeste_privadas = dados_nordeste_privadas[dados_nordeste_privadas["QE_I08_Convertido"] != "Não informado"]
order_q08_privadas = pd.DataFrame(dados_nordeste_privadas.groupby('QE_I08_Convertido')['NT_GER'].mean()).reset_index(
        ).sort_values(by='NT_GER', ascending=False)

layout = go.Layout(
    title=go.layout.Title(
        text='Desempenho de alunos de instituições públicas e privadas e suas caracteristicas socioeconômicas',
        x=0.5
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Média geral das notas',
            )
    ))

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
            x=order_q08_publicas["QE_I08_Convertido"],
            y=order_q08_publicas["NT_GER"],
            marker_color="#4169E1",
            name="Públicas"
    ))

fig.add_trace(go.Bar(
            x=order_q08_privadas["QE_I08_Convertido"],
            y=order_q08_privadas["NT_GER"],
            marker_color="#F08080",
            name="Privadas"
    ))
iplot(fig)

#### Conclusão:
Não, no gráfico podemos observar que alunos de universidades públicas se sairam melhores no quesito nota geral, mostrando então que estão saindo melhores profissionais, sendo mais direto, as universidades públicas formam melhores profissionais e observando a faixa socioeconômica vemos que isso não influência de maneira geral num desempenho melhor para os individuos na mesma faixa.

### Considerando cursos que tiveram o mesmo conceito no ENADE, os alunos compartilham características sócio-econômicas em comum? (essa é a pergunta mais trabalhosa para responder e cada grupo deve identificar como fazer isso).

In [6]:
dados_conceito = dados_nordeste.groupby(['CONCEITO', "QE_I08_Convertido"]).size().reset_index(name='count_conceito')
dados_conceito =  dados_conceito[dados_conceito["QE_I08_Convertido"] != "Não informado"]
conceito_1 = dados_conceito[dados_conceito["CONCEITO"]==1].sort_values(by='count_conceito', ascending=False)
conceito_2 = dados_conceito[dados_conceito["CONCEITO"]==2].sort_values(by='count_conceito', ascending=False)
conceito_3 = dados_conceito[dados_conceito["CONCEITO"]==3].sort_values(by='count_conceito', ascending=False)
conceito_4 = dados_conceito[dados_conceito["CONCEITO"]==4].sort_values(by='count_conceito', ascending=False)
conceito_5 = dados_conceito[dados_conceito["CONCEITO"]==5].sort_values(by='count_conceito', ascending=False)
trace1 =  go.Bar(
            x=conceito_1["QE_I08_Convertido"],
            y=conceito_1["count_conceito"],
            name="Conceito 1 - ENADE",
            marker_color="#F08080"
    )
trace2 = go.Bar(
            x=conceito_2["QE_I08_Convertido"],
            y=conceito_2["count_conceito"],
            name="Conceito 2 - ENADE",
            marker_color="#AFEEEE"
    )
trace3 =  go.Bar(
            x=conceito_3["QE_I08_Convertido"],
            y=conceito_3["count_conceito"],
            name="Conceito 3 - ENADE",
            marker_color="#D8BFD8"
    )
trace4 = go.Bar(
            x=conceito_4["QE_I08_Convertido"],
            y=conceito_4["count_conceito"],
            name="Conceito 4 - ENADE",
            marker_color="#FFC0CB"
    )
trace5 = go.Bar(
            x=conceito_5["QE_I08_Convertido"],
            y=conceito_5["count_conceito"],
            name="Conceito 5 - ENADE",
            marker_color="#4169E1"
    )
fig = tools.make_subplots(rows=1, cols=2, shared_yaxes=True)
fig1 = tools.make_subplots(rows=1, cols=2, shared_yaxes=True)
fig2 = tools.make_subplots(rows=1, cols=2, shared_yaxes=True)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig1.append_trace(trace3, 1, 1)
fig1.append_trace(trace4, 1, 2)
fig2.append_trace(trace5, 1, 1)


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



In [7]:
fig['layout'].update(title='Conceito 1 e 2 - Faixa Socioeconômica onde o aluno está inserido')
iplot(fig)

#### Conclusão:
**Conceito 1 ->** Podemos observar que os dados contem apenas candidatos que declaram renda familiar de **até 4,5 salários mínimos**, porém não conseguimos inferir muita coisa pela quantidade de dados que possuímos sobre o conceito 1 do ENADE.

**Conceito 2 ->** Sim possuem caracteristicas parecidas, já que mais de 50% declarou uma renda famíliar de até **4,5 salários minimos**.

In [8]:
fig1['layout'].update(title='Conceito 3 e 4 - Faixa Socioeconômica onde o aluno está inserido')
iplot(fig1)

#### Conclusão:
**Conceito 3 e 4 ->** Não possuem caracteristicas tão parecidas. Já que os dados estão distribuidos entre cada faixa de caracteristicas socioeconomicas numa quantidade bem similar.

In [9]:
fig2['layout'].update(title='Conceito 5 - Faixa Socioeconômica onde o aluno está inserido')
iplot(fig2)

#### Conclusão:
**Conceito 5 ->** Esse gráfico demonstra uma distribuição parecida entre as barras, ou seja, os alunos estão inseridos em contexto diferente em grande quantidade. Resumindo: Não possui caracteristicas semelhantes.

### De acordo com as características socioeconômicas do aluno relacionando com sua raça: Qual é o desempenho na prova do enade  Nota bruta geral total e se a diferença entre as minorias e os brancos


In [10]:
dados_nordeste_brancos = dados_nordeste[dados_nordeste["QE_I02"]=="A"]
dados_nordeste_brancos = dados_nordeste_brancos[dados_nordeste_brancos["QE_I08_Convertido"] != "Não informado"]
order_brancos = pd.DataFrame(dados_nordeste_brancos.groupby('QE_I08_Convertido')['NT_GER'].mean()).reset_index(
        ).sort_values(by='NT_GER', ascending=False)

dados_nordeste_afros = dados_nordeste[dados_nordeste["QE_I02"]=="B"]
dados_nordeste_afros = dados_nordeste_afros[dados_nordeste_afros["QE_I08_Convertido"] != "Não informado"]
order_afros = pd.DataFrame(dados_nordeste_afros.groupby('QE_I08_Convertido')['NT_GER'].mean()).reset_index(
        ).sort_values(by='NT_GER', ascending=False)

dados_nordeste_indigenas = dados_nordeste[dados_nordeste["QE_I02"]=="E"]
dados_nordeste_indigenas = dados_nordeste_indigenas[dados_nordeste_indigenas["QE_I08_Convertido"] != "Não informado"]
order_indigenas = pd.DataFrame(dados_nordeste_indigenas.groupby('QE_I08_Convertido')['NT_GER'].mean()).reset_index(
        ).sort_values(by='NT_GER', ascending=False)

layout = go.Layout(
    title=go.layout.Title(
        text='Desempenho de alunos de acordo com sua etnia',
        x=0.5
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Média geral das notas',
            )
    ))

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
            x=order_brancos["QE_I08_Convertido"],
            y=order_brancos["NT_GER"],
            marker_color="#4169E1",
            name="Brancos"
    ))

fig.add_trace(go.Bar(
            x=order_afros["QE_I08_Convertido"],
            y=order_afros["NT_GER"],
            marker_color="#F08080",
            name="Negros"
    ))
fig.add_trace(go.Bar(
            x=order_indigenas["QE_I08_Convertido"],
            y=order_indigenas["NT_GER"],
            marker_color="#B0C4DE",
            name="Indigenas"
    ))
iplot(fig)

#### Conclusão:
Geralmente alunos de etnia "branca" se sairam com notas melhores que os demais, tirando a faixa entre 1,5 salários-minimos. 

**Obs 1:** Ainda é bom observar que não se possui dados de etnia indigena que tenha declarado no questionário possuir mais do que 4,5 salários-minimos
**Obs 2:** O mesmo vale para negros mas a partir da faixa acima de 30 salários minimos.